In [2]:
import pandas as pd
import gensim
from gensim import models
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import numpy as np


In [3]:
train_data = pd.read_table('ratings_train.txt')

train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')
train_corpus = []

for index,row in train_data.iterrows():      
    tokens = gensim.utils.simple_preprocess(row['document'])         
    train_corpus.append(gensim.models.doc2vec.TaggedDocument(tokens,["{}_{}".format(row['label'],row['id'])]))
  

In [4]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=50)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [5]:
model.save('revie2vec')


In [6]:
test_data = pd.read_table('ratings_test.txt')


In [7]:
test_data = pd.read_table('ratings_test.txt')
test_data['document'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any')

correct = 0
for d,test in  test_data.iterrows():
    review_tokens = gensim.utils.simple_preprocess(test_data['document'][d])   
    inferred_vector = model.infer_vector(review_tokens)
    sims = model.docvecs.most_similar([inferred_vector], topn=10)
    predict = (int(sims[0][0][0]) +  int(sims[1][0][0]) +  int(sims[2][0][0]) ) / 3.0
    if(predict>0.5):
        predict = 1
    else:
        predict = 0
    if test_data['label'][d] == predict:
        correct = correct + 1

print("\n 테스트 정확도: %.4f" % (correct / len(test_data)))



 테스트 정확도: 0.6667
